# Using vectorized representations of text for semantic text similarity

This notebook has been created to allow students of the TNLP 25/26 course to complete their assignment on vectorized representations of text. This notebook is provided with the minimal information to start working on the assignment. Students will have to follow the [instructions](https://mespla.github.io/tpln2526/assignment-searchinvectorialspace/) of the assignment reflecting in this notebook the work done.

The starting point will be to install both the `scikit-learn` and the `sentence-embeddings` python libraries:

In [3]:
!pip install sentence-transformers scikit-learn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Once this is done, students will have to download and read the file containing the dataset consisting of a list of scientific paper's title and abstract.


In [4]:
# =========================
# Section 2 — Dataset Acquisition & Loading (EMNLP 2016–2018 JSON)
# =========================

import os, json, hashlib
import requests
from collections import Counter

DATA_URL = "https://sbert.net/datasets/emnlp2016-2018.json"
DATA_PATH = "emnlp2016-2018.json"   # Feel free to rename, but keep it consistent across the notebook
FORCE_DOWNLOAD = False             # Set True if you want to re-download even if the file exists

def sha256_of_file(path: str) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1024 * 1024), b""):
            h.update(chunk)
    return h.hexdigest()

def download_json(url: str, out_path: str, timeout: int = 120) -> None:
    r = requests.get(url, stream=True, timeout=timeout)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024 * 1024):
            if chunk:
                f.write(chunk)

# --- (1) Acquire the dataset file (download if missing) ---
if FORCE_DOWNLOAD or (not os.path.exists(DATA_PATH)):
    print(f"[Download] Fetching dataset from: {DATA_URL}")
    download_json(DATA_URL, DATA_PATH)
    print(f"[Download] Saved to: {DATA_PATH}")
else:
    print(f"[Cache] Using existing file: {DATA_PATH}")

# --- (2) Load JSON into memory ---
with open(DATA_PATH, "r", encoding="utf-8") as f:
    papers = json.load(f)

# Alias for compatibility with the starter template cells
data = papers

# --- (3) Validate expected JSON structure: root must be a list of dicts ---
if not isinstance(papers, list):
    raise TypeError(f"Expected JSON root to be a list, got: {type(papers)}")

if len(papers) == 0:
    raise ValueError("Dataset loaded but it is empty (N=0). File may be corrupted or not the expected dataset.")

if not isinstance(papers[0], dict):
    raise TypeError(f"Expected each item to be a dict, got first item type: {type(papers[0])}")

# --- (4) Provenance info (practical traceability) ---
file_size_mb = os.path.getsize(DATA_PATH) / (1024 * 1024)
print(f"\n[Provenance]")
print(f" - Source URL: {DATA_URL}")
print(f" - Local path: {DATA_PATH}")
print(f" - File size : {file_size_mb:.2f} MB")
print(f" - SHA-256   : {sha256_of_file(DATA_PATH)}")

# --- (5) Mandatory sanity checks ---
N = len(papers)
print(f"\n[Sanity checks]")
print(f" - N papers loaded: {N}")

required_keys = {"title", "abstract", "url", "venue", "year"}
missing_key_counts = Counter()

empty_title = 0
empty_abstract = 0
non_string_title = 0
non_string_abstract = 0

years = []
venues = []
urls = []

for p in papers:
    # Key presence
    for k in required_keys:
        if k not in p:
            missing_key_counts[k] += 1

    # Title / abstract existence + type + emptiness
    t = p.get("title", None)
    a = p.get("abstract", None)

    if not isinstance(t, str):
        non_string_title += 1
        t = "" if t is None else str(t)
    if not isinstance(a, str):
        non_string_abstract += 1
        a = "" if a is None else str(a)

    if len(t.strip()) == 0:
        empty_title += 1
    if len(a.strip()) == 0:
        empty_abstract += 1

    # Metadata distributions (for plausibility checks)
    venues.append(str(p.get("venue", "")).strip())
    urls.append(str(p.get("url", "")).strip())

    y = p.get("year", None)
    try:
        years.append(int(y))
    except Exception:
        years.append(None)

# Report key presence
if sum(missing_key_counts.values()) == 0:
    print(" - Key presence: OK (all required keys found in all records)")
else:
    print(" - Key presence issues:")
    for k in sorted(required_keys):
        if missing_key_counts[k] > 0:
            print(f"   * Missing '{k}': {missing_key_counts[k]} records")

# Report title/abstract health
print(f" - Empty titles   : {empty_title}")
print(f" - Empty abstracts: {empty_abstract}")
print(f" - Non-string titles   : {non_string_title}")
print(f" - Non-string abstracts: {non_string_abstract}")

# Year plausibility
valid_years = [y for y in years if isinstance(y, int)]
year_counts = Counter(valid_years)
print(f" - Year distribution (top): {year_counts.most_common(5)}")

outside = [y for y in valid_years if y < 2016 or y > 2018]
print(f" - Years outside 2016–2018: {len(outside)}")

# Venue plausibility
venue_counts = Counter([v for v in venues if v])
print(f" - Venue distribution (top): {venue_counts.most_common(5)}")

# Duplicate URLs (very practical duplicate detector)
url_counts = Counter([u for u in urls if u])
dupe_urls = sum(1 for u, c in url_counts.items() if c > 1)
print(f" - Duplicate URL keys: {dupe_urls} (unique non-empty URLs: {len(url_counts)})")

# --- (6) Access check (prove we can read title/abstract) ---
print(f"\n[Access check — first record]")
print(f" - title   : {papers[0].get('title','')[:120]}{'...' if len(papers[0].get('title',''))>120 else ''}")
print(f" - abstract: {papers[0].get('abstract','')[:200]}{'...' if len(papers[0].get('abstract',''))>200 else ''}")
print(f" - url     : {papers[0].get('url','')}")
print(f" - venue   : {papers[0].get('venue','')}")
print(f" - year    : {papers[0].get('year','')}")


[Cache] Using existing file: emnlp2016-2018.json

[Provenance]
 - Source URL: https://sbert.net/datasets/emnlp2016-2018.json
 - Local path: emnlp2016-2018.json
 - File size : 1.05 MB
 - SHA-256   : 9e6020503e5f0dd0e91dbb970d47f43c7621f67321249daed5990057e159961c

[Sanity checks]
 - N papers loaded: 974
 - Key presence: OK (all required keys found in all records)
 - Empty titles   : 0
 - Empty abstracts: 0
 - Non-string titles   : 0
 - Non-string abstracts: 0
 - Year distribution (top): [(2018, 549), (2017, 230), (2016, 195)]
 - Years outside 2016–2018: 0
 - Venue distribution (top): [('EMNLP', 974)]
 - Duplicate URL keys: 0 (unique non-empty URLs: 974)

[Access check — first record]
 - title   : Rule Extraction for Tree-to-Tree Transducers by Cost Minimization
 - abstract: Finite-state transducers give efficient representations of many Natural Language phenomena. They allow to account for complex lexicon restrictions encountered, without involving the use of a large set...
 - url     :

## Part 1:
From this point, students should be able to obtain BoW and TF-IDF representations of the dataset, and to obtain similar matches for the new scientific paper titles in the instructions of the exercise. Include here the code use to build the representations, as well as the discussion on the results obtained.

In [5]:
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


###Step 1: Preprocessing the Data



In [6]:
%%writefile section3_preprocessing.py
from __future__ import annotations

from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Sequence, Tuple


try:
    import pandas as pd
    _HAS_PANDAS = True
except Exception:
    pd = None  # type: ignore
    _HAS_PANDAS = False


@dataclass(frozen=True)
class PrepConfig:
    title_key: str = "title"
    abstract_key: str = "abstract"
    joiner: str = " "
    make_dataframe: bool = True


def _as_text(x: Any) -> str:
    """Minimal, non-aggressive coercion: keep text as-is (no casing/punct stripping)."""
    if x is None:
        return ""
    if isinstance(x, str):
        return x
    return str(x)


def build_paper_texts(
    records: Sequence[Dict[str, Any]],
    config: PrepConfig = PrepConfig(),
) -> Tuple[List[str], Optional["pd.DataFrame"]]:
    """
    Build the working 'text' field exactly as required:
        text = title + " " + abstract

    Document unit: one paper = one document.
    Returns:
      - paper_texts: list[str] with one concatenated document per paper
      - df (optional): pandas DataFrame including title/abstract/url/venue/year/text
    """
    if not isinstance(records, (list, tuple)):
        raise TypeError(f"records must be a sequence (list/tuple) of dicts, got {type(records)}")

    paper_texts: List[str] = []
    rows_for_df: List[Dict[str, Any]] = []

    for i, rec in enumerate(records):
        if not isinstance(rec, dict):
            raise TypeError(f"Each record must be a dict. Found {type(rec)} at index {i}.")

        title = _as_text(rec.get(config.title_key, ""))
        abstract = _as_text(rec.get(config.abstract_key, ""))

        # Exact construction rule (single space joiner)
        text = title + config.joiner + abstract

        paper_texts.append(text)

        if config.make_dataframe:
            rows_for_df.append(
                {
                    "title": title,
                    "abstract": abstract,
                    "url": rec.get("url", ""),
                    "venue": rec.get("venue", ""),
                    "year": rec.get("year", ""),
                    "text": text,
                }
            )

    df_out = None
    if config.make_dataframe:
        if not _HAS_PANDAS:
            raise ImportError("pandas is required for make_dataframe=True, but it is not available.")
        df_out = pd.DataFrame(rows_for_df)

    return paper_texts, df_out


def validate_paper_texts(paper_texts: Sequence[str], expected_n: int) -> None:
    """Sanity checks for Section 3 outputs."""
    if len(paper_texts) != expected_n:
        raise AssertionError(f"len(paper_texts)={len(paper_texts)} != expected_n={expected_n}")
    if any(not isinstance(t, str) for t in paper_texts):
        raise AssertionError("paper_texts must contain only strings.")
    if any(len(t) == 0 for t in paper_texts):
        raise AssertionError("paper_texts contains empty strings (unexpected for this dataset).")


Overwriting section3_preprocessing.py


In [7]:
# =========================
# Section 3 — Data Preparation for Vectorization (Part 1 – Step 1)
# =========================

from section3_preprocessing import PrepConfig, build_paper_texts, validate_paper_texts

# Use the variable created in Section 2 (you already set: data = papers)
records = data  # list of dicts, one paper per record

config = PrepConfig(
    title_key="title",
    abstract_key="abstract",
    joiner=" ",          # MUST be exactly one space
    make_dataframe=True  # optional, but very useful in a notebook
)

paper_texts, papers_df = build_paper_texts(records, config=config)

# Robust sanity checks
N = len(records)
validate_paper_texts(paper_texts, expected_n=N)

print("[Section 3] Preprocessing complete.")
print(f" - Document unit          : 1 paper = 1 document")
print(f" - N documents (paper_texts): {len(paper_texts)}")
print(f" - DataFrame created      : {papers_df is not None}")
print("\n[Section 3] Example document (first record):")
print(paper_texts[0][:400] + ("..." if len(paper_texts[0]) > 400 else ""))

# Optional: quick inspection (keeps later steps easier)
if papers_df is not None:
    display(papers_df.head(3))


[Section 3] Preprocessing complete.
 - Document unit          : 1 paper = 1 document
 - N documents (paper_texts): 974
 - DataFrame created      : True

[Section 3] Example document (first record):
Rule Extraction for Tree-to-Tree Transducers by Cost Minimization Finite-state transducers give efficient representations of many Natural Language phenomena. They allow to account for complex lexicon restrictions encountered, without involving the use of a large set of complex rules difficult to analyze. We here show that these representations can be made very compact, indicate how to perform the ...


,title,abstract,url,venue,year,text
0,Rule Extraction for Tree-to-Tree Transducers b...,Finite-state transducers give efficient repres...,http://aclweb.org/anthology/D16-1002,EMNLP,2016,Rule Extraction for Tree-to-Tree Transducers b...
1,A Neural Network for Coordination Boundary Pre...,We propose a neural-network based model for co...,http://aclweb.org/anthology/D16-1003,EMNLP,2016,A Neural Network for Coordination Boundary Pre...
2,"Distinguishing Past, On-going, and Future Even...",The tremendous amount of user generated data t...,http://aclweb.org/anthology/D16-1005,EMNLP,2016,"Distinguishing Past, On-going, and Future Even..."


In [8]:
%%writefile test_section3_preprocessing.py
import unittest

from section3_preprocessing import PrepConfig, build_paper_texts, validate_paper_texts


class TestSection3Preprocessing(unittest.TestCase):
    def test_basic_concatenation_rule(self):
        records = [
            {"title": "Hello", "abstract": "World", "url": "u", "venue": "v", "year": 2016},
            {"title": "A", "abstract": "B", "url": "u2", "venue": "v2", "year": 2017},
        ]
        cfg = PrepConfig(make_dataframe=False)
        paper_texts, df = build_paper_texts(records, cfg)
        self.assertIsNone(df)
        self.assertEqual(paper_texts, ["Hello World", "A B"])

    def test_stable_length_and_validation(self):
        records = [{"title": "T", "abstract": "X"} for _ in range(5)]
        cfg = PrepConfig(make_dataframe=False)
        paper_texts, _ = build_paper_texts(records, cfg)
        validate_paper_texts(paper_texts, expected_n=5)  # should not raise

    def test_type_coercion_is_minimal_and_safe(self):
        records = [{"title": 123, "abstract": None}]
        cfg = PrepConfig(make_dataframe=False)
        paper_texts, _ = build_paper_texts(records, cfg)
        # title becomes "123", abstract becomes ""
        self.assertEqual(paper_texts[0], "123 ")

    def test_rejects_non_dict_records(self):
        records = [{"title": "OK", "abstract": "OK"}, "not_a_dict"]
        cfg = PrepConfig(make_dataframe=False)
        with self.assertRaises(TypeError):
            build_paper_texts(records, cfg)

    def test_dataframe_creation_if_enabled(self):
        records = [{"title": "T", "abstract": "A", "url": "u", "venue": "EMNLP", "year": 2016}]
        cfg = PrepConfig(make_dataframe=True)
        paper_texts, df = build_paper_texts(records, cfg)
        self.assertIsNotNone(df)
        self.assertIn("text", df.columns)
        self.assertEqual(df.loc[0, "text"], paper_texts[0])


if __name__ == "__main__":
    unittest.main()


Overwriting test_section3_preprocessing.py


In [8]:
!python -m unittest -v test_section3_preprocessing.py


test_basic_concatenation_rule (test_section3_preprocessing.TestSection3Preprocessing.test_basic_concatenation_rule) ... ok
test_dataframe_creation_if_enabled (test_section3_preprocessing.TestSection3Preprocessing.test_dataframe_creation_if_enabled) ... ok
test_rejects_non_dict_records (test_section3_preprocessing.TestSection3Preprocessing.test_rejects_non_dict_records) ... ok
test_stable_length_and_validation (test_section3_preprocessing.TestSection3Preprocessing.test_stable_length_and_validation) ... ok
test_type_coercion_is_minimal_and_safe (test_section3_preprocessing.TestSection3Preprocessing.test_type_coercion_is_minimal_and_safe) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.002s

OK


###Step 2: Building the BoW and TF-IDF Representations

In [9]:
%%writefile section4_bow.py
from __future__ import annotations

from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Sequence, Tuple

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer


@dataclass(frozen=True)
class BoWConfig:
    """
    Configuration for a BoW (CountVectorizer) run.

    Required toggles for the assignment:
      - lowercase: ON vs OFF
      - stop_words: None vs 'english'
    """
    name: str
    lowercase: bool = True
    stop_words: Optional[str] = None
    max_features: Optional[int] = None
    ngram_range: Tuple[int, int] = (1, 1)


def _validate_texts(texts: Sequence[str]) -> None:
    if not isinstance(texts, (list, tuple)):
        raise TypeError(f"texts must be a list/tuple of strings, got {type(texts)}")
    if len(texts) == 0:
        raise ValueError("texts is empty (N=0).")
    if any(not isinstance(t, str) for t in texts):
        bad = next(type(t) for t in texts if not isinstance(t, str))
        raise TypeError(f"All texts must be strings. Found non-string: {bad}")
    if any(len(t) == 0 for t in texts):
        raise ValueError("texts contains empty strings (unexpected for this dataset).")


def build_bow_vectors(
    texts: Sequence[str],
    config: BoWConfig,
):
    """
    Fit a CountVectorizer and return:
      - vectorizer (fitted)
      - X (document-term matrix, sparse)
    """
    _validate_texts(texts)

    vectorizer = CountVectorizer(
        lowercase=config.lowercase,
        stop_words=config.stop_words,
        max_features=config.max_features,
        ngram_range=config.ngram_range,
    )
    X = vectorizer.fit_transform(texts)
    return vectorizer, X


def sparse_matrix_stats(X) -> Dict[str, Any]:
    """Compute basic inspection stats for a sparse doc-term matrix."""
    n_docs, n_vocab = X.shape
    nnz = int(X.nnz)
    total = int(n_docs) * int(n_vocab)
    density = float(nnz / total) if total > 0 else 0.0
    avg_nnz_per_doc = float(nnz / n_docs) if n_docs > 0 else 0.0

    return {
        "n_docs": int(n_docs),
        "n_vocab": int(n_vocab),
        "shape": (int(n_docs), int(n_vocab)),
        "nnz": nnz,
        "density": density,
        "avg_nnz_per_doc": avg_nnz_per_doc,
        "matrix_type": type(X).__name__,
    }


def feature_examples(vectorizer: CountVectorizer, n: int = 20) -> List[str]:
    feats = vectorizer.get_feature_names_out()
    return feats[:n].tolist()


def case_sensitive_stopword_survivors(vectorizer: CountVectorizer) -> int:
    """
    Detect the nuance: with lowercase=False and stop_words='english',
    capitalized variants of stopwords (e.g., 'The') can survive.

    Returns how many vocabulary terms are "stopwords after lowercasing"
    but are not exactly in the stopword list (case mismatch).
    """
    stop = vectorizer.get_stop_words()
    if not stop:
        return 0

    feats = vectorizer.get_feature_names_out()
    stop_set = set(stop)

    survivors = 0
    for f in feats:
        # survivors like "The" where f.lower() in stop_set but f not in stop_set
        fl = f.lower()
        if fl in stop_set and f not in stop_set:
            survivors += 1
    return survivors


def run_bow_grid(texts: Sequence[str]) -> Dict[str, Dict[str, Any]]:
    """
    Run the 2x2 grid required by the assignment:
      - lowercase ON/OFF
      - stopwords OFF/ON ('english')
    Returns a dict keyed by config name with vectorizer, X, and stats.
    """
    configs = [
        BoWConfig(name="bow_lc_on_sw_off", lowercase=True,  stop_words=None),
        BoWConfig(name="bow_lc_on_sw_on",  lowercase=True,  stop_words="english"),
        BoWConfig(name="bow_lc_off_sw_off",lowercase=False, stop_words=None),
        BoWConfig(name="bow_lc_off_sw_on", lowercase=False, stop_words="english"),
    ]

    results: Dict[str, Dict[str, Any]] = {}
    for cfg in configs:
        vect, X = build_bow_vectors(texts, cfg)
        stats = sparse_matrix_stats(X)
        survivors = case_sensitive_stopword_survivors(vect) if cfg.stop_words else 0

        results[cfg.name] = {
            "config": cfg,
            "vectorizer": vect,
            "X": X,
            "stats": stats,
            "case_sensitive_stopword_survivors": survivors,
        }

    return results


Overwriting section4_bow.py


In [10]:
# =========================
# Section 4 — BoW Vectorization Experiments (Part 1 – Step 2, BoW branch)
# =========================

import re
import numpy as np

from section4_bow import run_bow_grid, feature_examples

# Input from Section 3:
# - paper_texts: list[str] where each element is "title + ' ' + abstract"
texts = paper_texts

bow_results = run_bow_grid(texts)

# Deterministic order (avoids confusing output + makes comparison easier)
RUN_ORDER = ["bow_lc_on_sw_off", "bow_lc_on_sw_on", "bow_lc_off_sw_off", "bow_lc_off_sw_on"]

def vocab_letter_sample(vectorizer, n: int = 20):
    """Return the first n features that contain at least one alphabetic character."""
    feats = vectorizer.get_feature_names_out().tolist()
    letter_feats = [f for f in feats if re.search(r"[A-Za-z]", f)]
    return letter_feats[:n]

print("[Section 4] BoW experiments complete.")
print("Each run builds a sparse document-term matrix X with shape (N_documents, V_vocabulary).")
print("Each column is a token; each cell is the token count in that document.\n")

# Print compact inspection per configuration (stable ordering + hard sanity checks)
for name in RUN_ORDER:
    out = bow_results[name]
    cfg = out["config"]
    stats = out["stats"]
    survivors = out["case_sensitive_stopword_survivors"]

    # Robust invariants (fail fast if something is wrong)
    if stats["n_docs"] != len(texts):
        raise RuntimeError(
            f"Row mismatch in {name}: X has {stats['n_docs']} rows but texts has {len(texts)} documents"
        )
    if stats["n_vocab"] <= 0 or stats["nnz"] <= 0:
        raise RuntimeError(
            f"Degenerate BoW matrix in {name}: n_vocab={stats['n_vocab']}, nnz={stats['nnz']}"
        )

    print(f"--- {name} ---")
    print(f"Config: lowercase={cfg.lowercase}, stop_words={cfg.stop_words}")
    print(f"Matrix type : {stats['matrix_type']} (sparse)")
    print(f"Shape       : {stats['shape']}  (#docs x #vocab)")
    print(f"nnz         : {stats['nnz']}")
    print(f"Density     : {stats['density']:.6f}")
    print(f"Avg nnz/doc : {stats['avg_nnz_per_doc']:.2f}")

    # Stopword casing nuance indicator (only relevant when lowercase=False and stopwords ON)
    if cfg.stop_words is not None and cfg.lowercase is False:
        print(
            f"Stopword casing nuance: {survivors} vocab terms look like stopwords after lowercasing (e.g., 'The')"
        )

    # Two samples: (1) raw first tokens (often numeric), (2) a more meaningful sample with letters
    print(f"Vocabulary sample (first 20 terms): {feature_examples(out['vectorizer'], n=20)}")
    print(f"Vocabulary sample with letters (first 20): {vocab_letter_sample(out['vectorizer'], n=20)}\n")

# Optional: a small summary table (nice for the report)
try:
    import pandas as pd

    rows = []
    for name in RUN_ORDER:
        out = bow_results[name]
        cfg = out["config"]
        s = out["stats"]
        rows.append(
            {
                "run": name,
                "lowercase": cfg.lowercase,
                "stop_words": cfg.stop_words,
                "N_docs": s["n_docs"],
                "V_vocab": s["n_vocab"],
                "nnz": s["nnz"],
                "density": s["density"],
                "avg_nnz_per_doc": s["avg_nnz_per_doc"],
                "case_sensitive_stopword_survivors": out["case_sensitive_stopword_survivors"],
            }
        )

    bow_summary_df = pd.DataFrame(rows)
    bow_summary_df["run"] = pd.Categorical(bow_summary_df["run"], categories=RUN_ORDER, ordered=True)
    bow_summary_df = bow_summary_df.sort_values(by=["run"]).reset_index(drop=True)
    display(bow_summary_df)

except Exception:
    bow_summary_df = None

# Provide a default BoW representation for later sections if you want:
# (This prevents NameError later if you decide to use BoW first in Step 3.)
DEFAULT_BOW_RUN = "bow_lc_on_sw_off"
bow_vectorizer = bow_results[DEFAULT_BOW_RUN]["vectorizer"]
bow_paper_vectors = bow_results[DEFAULT_BOW_RUN]["X"]
print(f"[Section 4] Default BoW run set for later use: {DEFAULT_BOW_RUN}")


[Section 4] BoW experiments complete.
Each run builds a sparse document-term matrix X with shape (N_documents, V_vocabulary).
Each column is a token; each cell is the token count in that document.

--- bow_lc_on_sw_off ---
Config: lowercase=True, stop_words=None
Matrix type : csr_matrix (sparse)
Shape       : (974, 8404)  (#docs x #vocab)
nnz         : 90157
Density     : 0.011014
Avg nnz/doc : 92.56
Vocabulary sample (first 20 terms): ['000', '01', '02', '05', '050', '05365v2', '06', '06318v4', '09', '091', '094', '10', '100', '100k', '10m', '11', '110', '113k', '117', '118']
Vocabulary sample with letters (first 20): ['05365v2', '06318v4', '100k', '10m', '113k', '11x', '128x128', '12th', '12x', '14x', '1971a', '1a', '1d', '1k', '1st', '2005a', '2007b', '2013b', '2015a', '2016a']

--- bow_lc_on_sw_on ---
Config: lowercase=True, stop_words=english
Matrix type : csr_matrix (sparse)
Shape       : (974, 8153)  (#docs x #vocab)
nnz         : 64523
Density     : 0.008125
Avg nnz/doc : 66.25

,run,lowercase,stop_words,N_docs,V_vocab,nnz,density,avg_nnz_per_doc,case_sensitive_stopword_survivors
0,bow_lc_on_sw_off,True,None,974,8404,90157,0.011014,92.563655,0
1,bow_lc_on_sw_on,True,english,974,8153,64523,0.008125,66.245380,0
2,bow_lc_off_sw_off,False,None,974,10342,96257,0.009556,98.826489,0
3,bow_lc_off_sw_on,False,english,974,10094,72143,0.007338,74.068789,155


[Section 4] Default BoW run set for later use: bow_lc_on_sw_off


In [11]:
%%writefile test_all.py
import unittest

from section3_preprocessing import PrepConfig, build_paper_texts, validate_paper_texts
from section4_bow import (
    BoWConfig,
    build_bow_vectors,
    sparse_matrix_stats,
    case_sensitive_stopword_survivors,
)


class TestSection3Preprocessing(unittest.TestCase):
    def test_basic_concatenation_rule(self):
        records = [
            {"title": "Hello", "abstract": "World", "url": "u", "venue": "v", "year": 2016},
            {"title": "A", "abstract": "B", "url": "u2", "venue": "v2", "year": 2017},
        ]
        cfg = PrepConfig(make_dataframe=False)
        paper_texts, df = build_paper_texts(records, cfg)
        self.assertIsNone(df)
        self.assertEqual(paper_texts, ["Hello World", "A B"])

    def test_stable_length_and_validation(self):
        records = [{"title": "T", "abstract": "X"} for _ in range(5)]
        cfg = PrepConfig(make_dataframe=False)
        paper_texts, _ = build_paper_texts(records, cfg)
        validate_paper_texts(paper_texts, expected_n=5)  # should not raise

    def test_type_coercion_is_minimal_and_safe(self):
        records = [{"title": 123, "abstract": None}]
        cfg = PrepConfig(make_dataframe=False)
        paper_texts, _ = build_paper_texts(records, cfg)
        self.assertEqual(paper_texts[0], "123 ")

    def test_rejects_non_dict_records(self):
        records = [{"title": "OK", "abstract": "OK"}, "not_a_dict"]
        cfg = PrepConfig(make_dataframe=False)
        with self.assertRaises(TypeError):
            build_paper_texts(records, cfg)


class TestSection4BoW(unittest.TestCase):
    def test_build_bow_vectors_shape_and_sparse(self):
        texts = ["cat sat", "dog sat", "cat dog"]
        cfg = BoWConfig(name="t", lowercase=True, stop_words=None)
        vect, X = build_bow_vectors(texts, cfg)
        self.assertEqual(X.shape[0], 3)
        self.assertGreater(X.shape[1], 0)
        self.assertTrue(hasattr(X, "nnz"))  # sparse matrices expose nnz
        self.assertTrue(hasattr(vect, "vocabulary_"))

    def test_lowercase_merges_tokens(self):
        texts = ["BERT model", "bert model"]

        vect_off, X_off = build_bow_vectors(texts, BoWConfig(name="off", lowercase=False, stop_words=None))
        feats_off = set(vect_off.get_feature_names_out().tolist())
        self.assertIn("BERT", feats_off)
        self.assertIn("bert", feats_off)

        vect_on, X_on = build_bow_vectors(texts, BoWConfig(name="on", lowercase=True, stop_words=None))
        feats_on = set(vect_on.get_feature_names_out().tolist())
        self.assertNotIn("BERT", feats_on)
        self.assertIn("bert", feats_on)

        self.assertEqual(X_off.shape[0], X_on.shape[0])

    def test_stopwords_removed_when_lowercase_true(self):
        texts = ["the cat sat", "the dog sat"]

        vect_no, _ = build_bow_vectors(texts, BoWConfig(name="no", lowercase=True, stop_words=None))
        feats_no = set(vect_no.get_feature_names_out().tolist())
        self.assertIn("the", feats_no)

        vect_yes, _ = build_bow_vectors(texts, BoWConfig(name="yes", lowercase=True, stop_words="english"))
        feats_yes = set(vect_yes.get_feature_names_out().tolist())
        self.assertNotIn("the", feats_yes)

    def test_case_sensitive_stopword_survivors_detected(self):
        texts = ["The cat and the dog"]  # 'The' may survive when lowercase=False with english stopwords
        vect, _ = build_bow_vectors(texts, BoWConfig(name="cs", lowercase=False, stop_words="english"))
        survivors = case_sensitive_stopword_survivors(vect)
        self.assertGreaterEqual(survivors, 1)

    def test_sparse_stats_sane(self):
        texts = ["cat sat", "dog barked"]
        vect, X = build_bow_vectors(texts, BoWConfig(name="s", lowercase=True, stop_words=None))
        s = sparse_matrix_stats(X)
        self.assertEqual(s["n_docs"], 2)
        self.assertGreater(s["n_vocab"], 0)
        self.assertGreaterEqual(s["density"], 0.0)
        self.assertLessEqual(s["density"], 1.0)


if __name__ == "__main__":
    unittest.main(verbosity=2)


Overwriting test_all.py


In [12]:
!python -m unittest -v test_all.py


test_basic_concatenation_rule (test_all.TestSection3Preprocessing.test_basic_concatenation_rule) ... ok
test_rejects_non_dict_records (test_all.TestSection3Preprocessing.test_rejects_non_dict_records) ... ok
test_stable_length_and_validation (test_all.TestSection3Preprocessing.test_stable_length_and_validation) ... ok
test_type_coercion_is_minimal_and_safe (test_all.TestSection3Preprocessing.test_type_coercion_is_minimal_and_safe) ... ok
test_build_bow_vectors_shape_and_sparse (test_all.TestSection4BoW.test_build_bow_vectors_shape_and_sparse) ... ok
test_case_sensitive_stopword_survivors_detected (test_all.TestSection4BoW.test_case_sensitive_stopword_survivors_detected) ... ok
test_lowercase_merges_tokens (test_all.TestSection4BoW.test_lowercase_merges_tokens) ... ok
test_sparse_stats_sane (test_all.TestSection4BoW.test_sparse_stats_sane) ... ok
test_stopwords_removed_when_lowercase_true (test_all.TestSection4BoW.test_stopwords_removed_when_lowercase_true) ... ok

---------------------

In [17]:
%%writefile section5_tfidf.py
from __future__ import annotations

from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Sequence, Tuple

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


@dataclass(frozen=True)
class TfidfConfig:
    """
    Configuration for a TF-IDF (TfidfVectorizer) run.

    Required toggles for the assignment:
      - lowercase: ON vs OFF
      - stop_words: None vs 'english'
    """
    name: str
    lowercase: bool = True
    stop_words: Optional[str] = None
    max_features: Optional[int] = None
    ngram_range: Tuple[int, int] = (1, 1)

    # Keep sklearn defaults explicit for clarity in the report
    norm: Optional[str] = "l2"
    use_idf: bool = True
    smooth_idf: bool = True
    sublinear_tf: bool = False


def _validate_texts(texts: Sequence[str]) -> None:
    if not isinstance(texts, (list, tuple)):
        raise TypeError(f"texts must be a list/tuple of strings, got {type(texts)}")
    if len(texts) == 0:
        raise ValueError("texts is empty (N=0).")
    if any(not isinstance(t, str) for t in texts):
        bad = next(type(t) for t in texts if not isinstance(t, str))
        raise TypeError(f"All texts must be strings. Found non-string: {bad}")
    if any(len(t) == 0 for t in texts):
        raise ValueError("texts contains empty strings (unexpected for this dataset).")


def build_tfidf_vectors(
    texts: Sequence[str],
    config: TfidfConfig,
):
    """
    Fit a TfidfVectorizer and return:
      - vectorizer (fitted)
      - X (document-term TF-IDF matrix, sparse)
    """
    _validate_texts(texts)

    vectorizer = TfidfVectorizer(
        lowercase=config.lowercase,
        stop_words=config.stop_words,
        max_features=config.max_features,
        ngram_range=config.ngram_range,
        norm=config.norm,
        use_idf=config.use_idf,
        smooth_idf=config.smooth_idf,
        sublinear_tf=config.sublinear_tf,
    )
    X = vectorizer.fit_transform(texts)
    return vectorizer, X


def sparse_matrix_stats(X) -> Dict[str, Any]:
    """Compute basic inspection stats for a sparse doc-term matrix."""
    n_docs, n_vocab = X.shape
    nnz = int(X.nnz)
    total = int(n_docs) * int(n_vocab)
    density = float(nnz / total) if total > 0 else 0.0
    avg_nnz_per_doc = float(nnz / n_docs) if n_docs > 0 else 0.0

    return {
        "n_docs": int(n_docs),
        "n_vocab": int(n_vocab),
        "shape": (int(n_docs), int(n_vocab)),
        "nnz": nnz,
        "density": density,
        "avg_nnz_per_doc": avg_nnz_per_doc,
        "matrix_type": type(X).__name__,
        "dtype": str(getattr(X, "dtype", "unknown")),
    }


def feature_examples(vectorizer: TfidfVectorizer, n: int = 20) -> List[str]:
    feats = vectorizer.get_feature_names_out()
    return feats[:n].tolist()


def case_sensitive_stopword_survivors(vectorizer: TfidfVectorizer) -> int:
    """
    Detect the nuance: with lowercase=False and stop_words='english',
    capitalized variants of stopwords (e.g., 'The') can survive.
    """
    stop = vectorizer.get_stop_words()
    if not stop:
        return 0

    feats = vectorizer.get_feature_names_out()
    stop_set = set(stop)

    survivors = 0
    for f in feats:
        fl = f.lower()
        if fl in stop_set and f not in stop_set:
            survivors += 1
    return survivors


def l2_norm_sample_stats(X, sample_n: int = 50, seed: int = 0) -> Dict[str, float]:
    """
    TF-IDF vectors in sklearn are L2-normalized by default (norm='l2').
    This helper checks that empirically on a sample of rows.
    """
    n_docs = X.shape[0]
    if n_docs == 0:
        return {"min": 0.0, "mean": 0.0, "max": 0.0}

    sample_n = int(min(sample_n, n_docs))
    rng = np.random.default_rng(seed)
    idx = rng.choice(n_docs, size=sample_n, replace=False)

    Xs = X[idx]
    norms = np.sqrt(np.asarray(Xs.multiply(Xs).sum(axis=1)).ravel())

    return {
        "min": float(np.min(norms)),
        "mean": float(np.mean(norms)),
        "max": float(np.max(norms)),
    }


def top_terms_for_doc(
    X,
    vectorizer: TfidfVectorizer,
    doc_index: int,
    top_n: int = 10,
) -> List[Tuple[str, float]]:
    """Return top-N (term, tf-idf weight) pairs for one document row."""
    feats = vectorizer.get_feature_names_out()
    row = X.getrow(doc_index)
    if row.nnz == 0:
        return []

    order = np.argsort(row.data)[::-1][:top_n]
    return [(feats[row.indices[i]], float(row.data[i])) for i in order]


def run_tfidf_grid(texts: Sequence[str], sample_n_for_norms: int = 50) -> Dict[str, Dict[str, Any]]:
    """
    Run the 2x2 grid required by the assignment:
      - lowercase ON/OFF
      - stopwords OFF/ON ('english')
    """
    configs = [
        TfidfConfig(name="tfidf_lc_on_sw_off",  lowercase=True,  stop_words=None),
        TfidfConfig(name="tfidf_lc_on_sw_on",   lowercase=True,  stop_words="english"),
        TfidfConfig(name="tfidf_lc_off_sw_off", lowercase=False, stop_words=None),
        TfidfConfig(name="tfidf_lc_off_sw_on",  lowercase=False, stop_words="english"),
    ]

    results: Dict[str, Dict[str, Any]] = {}
    for cfg in configs:
        vect, X = build_tfidf_vectors(texts, cfg)
        stats = sparse_matrix_stats(X)
        survivors = case_sensitive_stopword_survivors(vect) if cfg.stop_words else 0
        norms = l2_norm_sample_stats(X, sample_n=sample_n_for_norms, seed=0)

        results[cfg.name] = {
            "config": cfg,
            "vectorizer": vect,
            "X": X,
            "stats": stats,
            "l2_norm_sample": norms,
            "case_sensitive_stopword_survivors": survivors,
        }

    return results


Overwriting section5_tfidf.py


In [18]:
# =========================
# Section 5 — TF-IDF Vectorization Experiments (Part 1 – Step 2, TF-IDF branch)
# =========================

import re
import numpy as np

from section5_tfidf import run_tfidf_grid, feature_examples, top_terms_for_doc

# Input from Section 3:
texts = paper_texts

tfidf_results = run_tfidf_grid(texts, sample_n_for_norms=100)

RUN_ORDER_TFIDF = ["tfidf_lc_on_sw_off", "tfidf_lc_on_sw_on", "tfidf_lc_off_sw_off", "tfidf_lc_off_sw_on"]

def vocab_letter_sample(vectorizer, n: int = 20):
    feats = vectorizer.get_feature_names_out().tolist()
    letter_feats = [f for f in feats if re.search(r"[A-Za-z]", f)]
    return letter_feats[:n]

print("[Section 5] TF-IDF experiments complete.")
print("Each run builds a sparse document-term matrix X with shape (N_documents, V_vocabulary).")
print("Each column is a token; each cell is a TF-IDF weight (NOT a raw count).")
print("Note: sklearn TF-IDF uses L2-normalization by default, so ||doc_vector||_2 ≈ 1.\n")

for name in RUN_ORDER_TFIDF:
    out = tfidf_results[name]
    cfg = out["config"]
    s = out["stats"]
    norms = out["l2_norm_sample"]
    survivors = out["case_sensitive_stopword_survivors"]

    # Hard invariants
    if s["n_docs"] != len(texts):
        raise RuntimeError(f"Row mismatch in {name}: X has {s['n_docs']} rows but texts has {len(texts)} docs")
    if s["n_vocab"] <= 0 or s["nnz"] <= 0:
        raise RuntimeError(f"Degenerate TF-IDF matrix in {name}: n_vocab={s['n_vocab']}, nnz={s['nnz']}")

    print(f"--- {name} ---")
    print(f"Config: lowercase={cfg.lowercase}, stop_words={cfg.stop_words}")
    print(f"Matrix type : {s['matrix_type']} (sparse)")
    print(f"Dtype       : {s['dtype']} (TF-IDF weights are floating-point)")
    print(f"Shape       : {s['shape']}  (#docs x #vocab)")
    print(f"nnz         : {s['nnz']}")
    print(f"Density     : {s['density']:.6f}")
    print(f"Avg nnz/doc : {s['avg_nnz_per_doc']:.2f}")
    print(f"L2-norm sample (min/mean/max): {norms['min']:.4f} / {norms['mean']:.4f} / {norms['max']:.4f}")

    if cfg.stop_words is not None and cfg.lowercase is False:
        print(f"Stopword casing nuance: {survivors} vocab terms look like stopwords after lowercasing (e.g., 'The')")

    print(f"Vocabulary sample (first 20 terms): {feature_examples(out['vectorizer'], n=20)}")
    print(f"Vocabulary sample with letters (first 20): {vocab_letter_sample(out['vectorizer'], n=20)}")

    # Show top TF-IDF terms for the first document as a concrete interpretability check
    top_terms = top_terms_for_doc(out["X"], out["vectorizer"], doc_index=0, top_n=10)
    print("Top TF-IDF terms (doc 0):")
    for term, val in top_terms:
        print(f" - {term:20s}  {val:.4f}")
    print()

# Summary table (nice for reporting)
try:
    import pandas as pd

    rows = []
    for name in RUN_ORDER_TFIDF:
        out = tfidf_results[name]
        cfg = out["config"]
        s = out["stats"]
        norms = out["l2_norm_sample"]
        rows.append(
            {
                "run": name,
                "lowercase": cfg.lowercase,
                "stop_words": cfg.stop_words,
                "N_docs": s["n_docs"],
                "V_vocab": s["n_vocab"],
                "nnz": s["nnz"],
                "density": s["density"],
                "avg_nnz_per_doc": s["avg_nnz_per_doc"],
                "l2_norm_mean(sample)": norms["mean"],
                "case_sensitive_stopword_survivors": out["case_sensitive_stopword_survivors"],
            }
        )

    tfidf_summary_df = pd.DataFrame(rows)
    tfidf_summary_df["run"] = pd.Categorical(tfidf_summary_df["run"], categories=RUN_ORDER_TFIDF, ordered=True)
    tfidf_summary_df = tfidf_summary_df.sort_values(by=["run"]).reset_index(drop=True)
    display(tfidf_summary_df)

except Exception:
    tfidf_summary_df = None

# -------------------------
# BoW vs TF-IDF comparison (interpretability-friendly, inside Section 5)
# -------------------------
# We compare (1) values meaning (counts vs weights) and (2) a concrete example on doc 0.

def top_terms_bow_counts(X, vectorizer, doc_index: int, top_n: int = 10):
    feats = vectorizer.get_feature_names_out()
    row = X.getrow(doc_index)
    if row.nnz == 0:
        return []
    order = np.argsort(row.data)[::-1][:top_n]
    return [(feats[row.indices[i]], int(row.data[i])) for i in order]

DEFAULT_TFIDF_RUN = "tfidf_lc_on_sw_off"
tfidf_vectorizer = tfidf_results[DEFAULT_TFIDF_RUN]["vectorizer"]
tfidf_paper_vectors = tfidf_results[DEFAULT_TFIDF_RUN]["X"]

print(f"[Section 5] Default TF-IDF run set for later use: {DEFAULT_TFIDF_RUN}")

# If you already computed BoW in Section 4, these variables exist:
#   bow_vectorizer, bow_paper_vectors
if "bow_vectorizer" in globals() and "bow_paper_vectors" in globals():
    print("\n[Section 5] BoW vs TF-IDF (doc 0 example)")
    bow_top = top_terms_bow_counts(bow_paper_vectors, bow_vectorizer, doc_index=0, top_n=10)
    tfidf_top = top_terms_for_doc(tfidf_paper_vectors, tfidf_vectorizer, doc_index=0, top_n=10)

    print("Top BoW terms by COUNT (doc 0):")
    for term, cnt in bow_top:
        print(f" - {term:20s}  {cnt}")

    print("\nTop TF-IDF terms by WEIGHT (doc 0):")
    for term, w in tfidf_top:
        print(f" - {term:20s}  {w:.4f}")


[Section 5] TF-IDF experiments complete.
Each run builds a sparse document-term matrix X with shape (N_documents, V_vocabulary).
Each column is a token; each cell is a TF-IDF weight (NOT a raw count).
Note: sklearn TF-IDF uses L2-normalization by default, so ||doc_vector||_2 ≈ 1.

--- tfidf_lc_on_sw_off ---
Config: lowercase=True, stop_words=None
Matrix type : csr_matrix (sparse)
Dtype       : float64 (TF-IDF weights are floating-point)
Shape       : (974, 8404)  (#docs x #vocab)
nnz         : 90157
Density     : 0.011014
Avg nnz/doc : 92.56
L2-norm sample (min/mean/max): 1.0000 / 1.0000 / 1.0000
Vocabulary sample (first 20 terms): ['000', '01', '02', '05', '050', '05365v2', '06', '06318v4', '09', '091', '094', '10', '100', '100k', '10m', '11', '110', '113k', '117', '118']
Vocabulary sample with letters (first 20): ['05365v2', '06318v4', '100k', '10m', '113k', '11x', '128x128', '12th', '12x', '14x', '1971a', '1a', '1d', '1k', '1st', '2005a', '2007b', '2013b', '2015a', '2016a']
Top TF-I

,run,lowercase,stop_words,N_docs,V_vocab,nnz,density,avg_nnz_per_doc,l2_norm_mean(sample),case_sensitive_stopword_survivors
0,tfidf_lc_on_sw_off,True,None,974,8404,90157,0.011014,92.563655,1.0,0
1,tfidf_lc_on_sw_on,True,english,974,8153,64523,0.008125,66.245380,1.0,0
2,tfidf_lc_off_sw_off,False,None,974,10342,96257,0.009556,98.826489,1.0,0
3,tfidf_lc_off_sw_on,False,english,974,10094,72143,0.007338,74.068789,1.0,155


[Section 5] Default TF-IDF run set for later use: tfidf_lc_on_sw_off

[Section 5] BoW vs TF-IDF (doc 0 example)
Top BoW terms by COUNT (doc 0):
 - of                    4
 - to                    4
 - tree                  2
 - for                   2
 - minimization          2
 - representations       2
 - complex               2
 - the                   2
 - transducers           2
 - operation             1

Top TF-IDF terms by WEIGHT (doc 0):
 - transducers           0.3709
 - minimization          0.3430
 - tree                  0.2335
 - complex               0.2053
 - restrictions          0.1854
 - encountered           0.1715
 - representations       0.1600
 - compact               0.1525
 - involving             0.1499
 - finite                0.1475


In [19]:
%%writefile test_all.py
import unittest
import numpy as np

from section3_preprocessing import PrepConfig, build_paper_texts, validate_paper_texts

from section4_bow import (
    BoWConfig,
    build_bow_vectors,
    sparse_matrix_stats as bow_sparse_matrix_stats,
    case_sensitive_stopword_survivors as bow_case_sensitive_stopword_survivors,
)

from section5_tfidf import (
    TfidfConfig,
    build_tfidf_vectors,
    sparse_matrix_stats as tfidf_sparse_matrix_stats,
    case_sensitive_stopword_survivors as tfidf_case_sensitive_stopword_survivors,
    l2_norm_sample_stats,
    top_terms_for_doc,
)


class TestSection3Preprocessing(unittest.TestCase):
    def test_basic_concatenation_rule(self):
        records = [
            {"title": "Hello", "abstract": "World", "url": "u", "venue": "v", "year": 2016},
            {"title": "A", "abstract": "B", "url": "u2", "venue": "v2", "year": 2017},
        ]
        cfg = PrepConfig(make_dataframe=False)
        paper_texts, df = build_paper_texts(records, cfg)
        self.assertIsNone(df)
        self.assertEqual(paper_texts, ["Hello World", "A B"])

    def test_stable_length_and_validation(self):
        records = [{"title": "T", "abstract": "X"} for _ in range(5)]
        cfg = PrepConfig(make_dataframe=False)
        paper_texts, _ = build_paper_texts(records, cfg)
        validate_paper_texts(paper_texts, expected_n=5)  # should not raise

    def test_type_coercion_is_minimal_and_safe(self):
        records = [{"title": 123, "abstract": None}]
        cfg = PrepConfig(make_dataframe=False)
        paper_texts, _ = build_paper_texts(records, cfg)
        self.assertEqual(paper_texts[0], "123 ")

    def test_rejects_non_dict_records(self):
        records = [{"title": "OK", "abstract": "OK"}, "not_a_dict"]
        cfg = PrepConfig(make_dataframe=False)
        with self.assertRaises(TypeError):
            build_paper_texts(records, cfg)


class TestSection4BoW(unittest.TestCase):
    def test_build_bow_vectors_shape_and_sparse(self):
        texts = ["cat sat", "dog sat", "cat dog"]
        cfg = BoWConfig(name="t", lowercase=True, stop_words=None)
        vect, X = build_bow_vectors(texts, cfg)
        self.assertEqual(X.shape[0], 3)
        self.assertGreater(X.shape[1], 0)
        self.assertTrue(hasattr(X, "nnz"))
        self.assertTrue(hasattr(vect, "vocabulary_"))

    def test_lowercase_merges_tokens(self):
        texts = ["BERT model", "bert model"]

        vect_off, _ = build_bow_vectors(texts, BoWConfig(name="off", lowercase=False, stop_words=None))
        feats_off = set(vect_off.get_feature_names_out().tolist())
        self.assertIn("BERT", feats_off)
        self.assertIn("bert", feats_off)

        vect_on, _ = build_bow_vectors(texts, BoWConfig(name="on", lowercase=True, stop_words=None))
        feats_on = set(vect_on.get_feature_names_out().tolist())
        self.assertNotIn("BERT", feats_on)
        self.assertIn("bert", feats_on)

    def test_stopwords_removed_when_lowercase_true(self):
        texts = ["the cat sat", "the dog sat"]

        vect_no, _ = build_bow_vectors(texts, BoWConfig(name="no", lowercase=True, stop_words=None))
        feats_no = set(vect_no.get_feature_names_out().tolist())
        self.assertIn("the", feats_no)

        vect_yes, _ = build_bow_vectors(texts, BoWConfig(name="yes", lowercase=True, stop_words="english"))
        feats_yes = set(vect_yes.get_feature_names_out().tolist())
        self.assertNotIn("the", feats_yes)

    def test_case_sensitive_stopword_survivors_detected(self):
        texts = ["The cat and the dog"]
        vect, _ = build_bow_vectors(texts, BoWConfig(name="cs", lowercase=False, stop_words="english"))
        survivors = bow_case_sensitive_stopword_survivors(vect)
        self.assertGreaterEqual(survivors, 1)

    def test_sparse_stats_sane(self):
        texts = ["cat sat", "dog barked"]
        _, X = build_bow_vectors(texts, BoWConfig(name="s", lowercase=True, stop_words=None))
        s = bow_sparse_matrix_stats(X)
        self.assertEqual(s["n_docs"], 2)
        self.assertGreater(s["n_vocab"], 0)
        self.assertGreaterEqual(s["density"], 0.0)
        self.assertLessEqual(s["density"], 1.0)


class TestSection5Tfidf(unittest.TestCase):
    def test_build_tfidf_vectors_shape_sparse_and_float(self):
        texts = ["cat sat", "dog sat", "cat dog"]
        cfg = TfidfConfig(name="t", lowercase=True, stop_words=None)
        vect, X = build_tfidf_vectors(texts, cfg)
        self.assertEqual(X.shape[0], 3)
        self.assertGreater(X.shape[1], 0)
        self.assertTrue(hasattr(X, "nnz"))
        self.assertTrue("float" in str(X.dtype))

        stats = tfidf_sparse_matrix_stats(X)
        self.assertEqual(stats["n_docs"], 3)
        self.assertGreater(stats["n_vocab"], 0)

    def test_tfidf_lowercase_merges_tokens(self):
        texts = ["BERT model", "bert model"]

        vect_off, _ = build_tfidf_vectors(texts, TfidfConfig(name="off", lowercase=False, stop_words=None))
        feats_off = set(vect_off.get_feature_names_out().tolist())
        self.assertIn("BERT", feats_off)
        self.assertIn("bert", feats_off)

        vect_on, _ = build_tfidf_vectors(texts, TfidfConfig(name="on", lowercase=True, stop_words=None))
        feats_on = set(vect_on.get_feature_names_out().tolist())
        self.assertNotIn("BERT", feats_on)
        self.assertIn("bert", feats_on)

    def test_tfidf_stopwords_removed_when_lowercase_true(self):
        texts = ["the cat sat", "the dog sat"]

        vect_no, _ = build_tfidf_vectors(texts, TfidfConfig(name="no", lowercase=True, stop_words=None))
        feats_no = set(vect_no.get_feature_names_out().tolist())
        self.assertIn("the", feats_no)

        vect_yes, _ = build_tfidf_vectors(texts, TfidfConfig(name="yes", lowercase=True, stop_words="english"))
        feats_yes = set(vect_yes.get_feature_names_out().tolist())
        self.assertNotIn("the", feats_yes)

    def test_tfidf_case_sensitive_stopword_survivors_detected(self):
        texts = ["The cat and the dog"]
        vect, _ = build_tfidf_vectors(texts, TfidfConfig(name="cs", lowercase=False, stop_words="english"))
        survivors = tfidf_case_sensitive_stopword_survivors(vect)
        self.assertGreaterEqual(survivors, 1)

    def test_tfidf_l2_norms_are_about_one(self):
        texts = ["cat sat", "dog sat", "cat dog"]
        vect, X = build_tfidf_vectors(texts, TfidfConfig(name="n", lowercase=True, stop_words=None))
        norms = l2_norm_sample_stats(X, sample_n=3, seed=0)
        self.assertAlmostEqual(norms["mean"], 1.0, places=6)

    def test_tfidf_idf_downweights_common_terms_when_tf_equal(self):
        # common appears in all docs; rare appears in only one doc => rare should get higher weight (tf equal)
        texts = ["common rare1", "common rare2"]
        vect, X = build_tfidf_vectors(texts, TfidfConfig(name="idf", lowercase=True, stop_words=None))
        feats = vect.get_feature_names_out().tolist()

        i_common = feats.index("common")
        i_rare1 = feats.index("rare1")

        row0 = X.getrow(0).toarray().ravel()
        self.assertGreater(row0[i_rare1], row0[i_common])

    def test_top_terms_for_doc_returns_sorted(self):
        texts = ["cat sat sat", "dog sat"]
        vect, X = build_tfidf_vectors(texts, TfidfConfig(name="top", lowercase=True, stop_words=None))
        top = top_terms_for_doc(X, vect, doc_index=0, top_n=5)
        self.assertTrue(len(top) > 0)
        # weights should be non-increasing
        weights = [w for _, w in top]
        self.assertTrue(all(weights[i] >= weights[i+1] for i in range(len(weights)-1)))


if __name__ == "__main__":
    unittest.main(verbosity=2)


Overwriting test_all.py


In [20]:
!python -m unittest -v test_all.py


test_basic_concatenation_rule (test_all.TestSection3Preprocessing.test_basic_concatenation_rule) ... ok
test_rejects_non_dict_records (test_all.TestSection3Preprocessing.test_rejects_non_dict_records) ... ok
test_stable_length_and_validation (test_all.TestSection3Preprocessing.test_stable_length_and_validation) ... ok
test_type_coercion_is_minimal_and_safe (test_all.TestSection3Preprocessing.test_type_coercion_is_minimal_and_safe) ... ok
test_build_bow_vectors_shape_and_sparse (test_all.TestSection4BoW.test_build_bow_vectors_shape_and_sparse) ... ok
test_case_sensitive_stopword_survivors_detected (test_all.TestSection4BoW.test_case_sensitive_stopword_survivors_detected) ... ok
test_lowercase_merges_tokens (test_all.TestSection4BoW.test_lowercase_merges_tokens) ... ok
test_sparse_stats_sane (test_all.TestSection4BoW.test_sparse_stats_sane) ... ok
test_stopwords_removed_when_lowercase_true (test_all.TestSection4BoW.test_stopwords_removed_when_lowercase_true) ... ok
test_build_tfidf_vecto

###Step 3: Similarity Search

In [1]:
%%writefile section6_queries.py
from __future__ import annotations

from dataclasses import dataclass
from typing import List, Sequence


@dataclass(frozen=True)
class QueryItem:
    label: str
    title: str
    abstract: str
    text: str


def _as_text(x) -> str:
    if x is None:
        return ""
    return x if isinstance(x, str) else str(x)


def build_query_item(label: str, title: str, abstract: str, joiner: str = " ") -> QueryItem:
    """
    Query construction MUST match dataset documents (Section 3):
        text = title + " " + abstract
    No extra cleaning here.
    """
    if not isinstance(joiner, str) or joiner != " ":
        raise ValueError('joiner must be exactly one space: " "')

    t = _as_text(title)
    a = _as_text(abstract)

    if len(t.strip()) == 0:
        raise ValueError(f"{label}: title is empty")
    if len(a.strip()) == 0:
        raise ValueError(f"{label}: abstract is empty")

    text = t + joiner + a
    return QueryItem(label=label, title=t, abstract=a, text=text)


def get_default_queries(joiner: str = " ") -> List[QueryItem]:
    """
    The three queries EXACTLY as provided by the assignment (title + abstract).
    Stored in fixed order: Query 1, Query 2, Query 3.
    """
    q1_title = "QUALES: Machine translation quality estimation via supervised and unsupervised machine learning."
    q1_abstract = (
        "The automatic quality estimation (QE) of machine translation consists in measuring the quality of translations "
        "without access to human references, usually via machine learning approaches. A good QE system can help in three "
        "aspects of translation processes involving machine translation and post-editing: increasing productivity (by ruling "
        "out poor quality machine translation), estimating costs (by helping to forecast the cost of post-editing) and selecting "
        "a provider (if several machine translation systems are available). Interest in this research area has grown significantly "
        "in recent years, leading to regular shared tasks in the main machine translation conferences and intense scientific activity. "
        "In this article we review the state of the art in this research area and present project QUALES, which is under development."
    )

    q2_title = "Learning to Ask Unanswerable Questions for Machine Reading Comprehension"
    q2_abstract = (
        "Machine reading comprehension with unanswerable questions is a challenging task. In this work, we propose a data augmentation "
        "technique by automatically generating relevant unanswerable questions according to an answerable question paired with its corresponding "
        "paragraph that contains the answer. We introduce a pair-to-sequence model for unanswerable question generation, which effectively captures "
        "the interactions between the question and the paragraph. We also present a way to construct training data for our question generation models "
        "by leveraging the existing reading comprehension dataset. Experimental results show that the pair-to-sequence model performs consistently better "
        "compared with the sequence-to-sequence baseline. We further use the automatically generated unanswerable questions as a means of data augmentation "
        "on the SQuAD 2.0 dataset, yielding 1.9 absolute F1 improvement with BERT-base model and 1.7 absolute F1 improvement with BERT-large model."
    )

    q3_title = "Unsupervised Neural Text Simplification"
    q3_abstract = (
        "The paper presents a first attempt towards unsupervised neural text simplification that relies only on unlabelled text corpora. "
        "The core framework is composed of a shared encoder and a pair of attentional-decoders, crucially assisted by discrimination-based losses and denoising. "
        "The framework is trained using unlabelled text collected from en-Wikipedia dump. Our analysis (both quantitative and qualitative involving human evaluators) "
        "on public test data shows that the proposed model can perform text-simplification at both lexical and syntactic levels, competitive to existing supervised methods. "
        "It also outperforms viable unsupervised baselines. Adding a few labelled pairs helps improve the performance further."
    )

    queries = [
        build_query_item("Query 1", q1_title, q1_abstract, joiner=joiner),
        build_query_item("Query 2", q2_title, q2_abstract, joiner=joiner),
        build_query_item("Query 3", q3_title, q3_abstract, joiner=joiner),
    ]
    return queries


def get_query_texts(queries: Sequence[QueryItem]) -> List[str]:
    return [q.text for q in queries]


def validate_queries(queries: Sequence[QueryItem]) -> None:
    if not isinstance(queries, (list, tuple)):
        raise TypeError(f"queries must be a list/tuple, got {type(queries)}")
    if len(queries) != 3:
        raise AssertionError(f"Expected exactly 3 queries, got {len(queries)}")

    expected_labels = ["Query 1", "Query 2", "Query 3"]
    got_labels = [q.label for q in queries]
    if got_labels != expected_labels:
        raise AssertionError(f"Query order/labels must be {expected_labels}, got {got_labels}")

    for q in queries:
        if any(not isinstance(x, str) for x in [q.label, q.title, q.abstract, q.text]):
            raise AssertionError("All QueryItem fields must be strings.")
        if len(q.title.strip()) == 0 or len(q.abstract.strip()) == 0 or len(q.text.strip()) == 0:
            raise AssertionError(f"{q.label} has empty fields.")

        # Strict pipeline check: text must be EXACTLY title + " " + abstract
        if q.text != (q.title + " " + q.abstract):
            raise AssertionError(f"{q.label}: text is not exactly title + ' ' + abstract")


Overwriting section6_queries.py


In [5]:
# =========================
# Section 6 — Query Set Definition (Part 1 – Step 3 Inputs)
# =========================

from section6_queries import get_default_queries, get_query_texts, validate_queries

queries = get_default_queries(joiner=" ")   # MUST be exactly one space
validate_queries(queries)

query_texts = get_query_texts(queries)      # list[str] in fixed order (Query 1, 2, 3)
query_labels = [q.label for q in queries]   # ["Query 1", "Query 2", "Query 3"]

print("[Section 6] Queries ready.")
print(f" - N queries     : {len(queries)}")
print(f" - Order/labels  : {query_labels}")
print("\n[Section 6] Preview (first 300 chars each):")
for q in queries:
    print(f"\n{q.label}: {q.title}")
    print(q.text[:300] + ("..." if len(q.text) > 300 else ""))


[Section 6] Queries ready.
 - N queries     : 3
 - Order/labels  : ['Query 1', 'Query 2', 'Query 3']

[Section 6] Preview (first 300 chars each):

Query 1: QUALES: Machine translation quality estimation via supervised and unsupervised machine learning.
QUALES: Machine translation quality estimation via supervised and unsupervised machine learning. The automatic quality estimation (QE) of machine translation consists in measuring the quality of translations without access to human references, usually via machine learning approaches. A good QE syste...

Query 2: Learning to Ask Unanswerable Questions for Machine Reading Comprehension
Learning to Ask Unanswerable Questions for Machine Reading Comprehension Machine reading comprehension with unanswerable questions is a challenging task. In this work, we propose a data augmentation technique by automatically generating relevant unanswerable questions according to an answerable ques...

Query 3: Unsupervised Neural Text Simplification
Unsupe

In [3]:
%%writefile test_all.py
import unittest
import numpy as np

from section3_preprocessing import PrepConfig, build_paper_texts, validate_paper_texts

from section4_bow import (
    BoWConfig,
    build_bow_vectors,
    sparse_matrix_stats as bow_sparse_matrix_stats,
    case_sensitive_stopword_survivors as bow_case_sensitive_stopword_survivors,
)

from section5_tfidf import (
    TfidfConfig,
    build_tfidf_vectors,
    sparse_matrix_stats as tfidf_sparse_matrix_stats,
    case_sensitive_stopword_survivors as tfidf_case_sensitive_stopword_survivors,
    l2_norm_sample_stats,
    top_terms_for_doc,
)

from section6_queries import (
    QueryItem,
    build_query_item,
    get_default_queries,
    get_query_texts,
    validate_queries,
)


class TestSection3Preprocessing(unittest.TestCase):
    def test_basic_concatenation_rule(self):
        records = [
            {"title": "Hello", "abstract": "World", "url": "u", "venue": "v", "year": 2016},
            {"title": "A", "abstract": "B", "url": "u2", "venue": "v2", "year": 2017},
        ]
        cfg = PrepConfig(make_dataframe=False)
        paper_texts, df = build_paper_texts(records, cfg)
        self.assertIsNone(df)
        self.assertEqual(paper_texts, ["Hello World", "A B"])

    def test_stable_length_and_validation(self):
        records = [{"title": "T", "abstract": "X"} for _ in range(5)]
        cfg = PrepConfig(make_dataframe=False)
        paper_texts, _ = build_paper_texts(records, cfg)
        validate_paper_texts(paper_texts, expected_n=5)

    def test_type_coercion_is_minimal_and_safe(self):
        records = [{"title": 123, "abstract": None}]
        cfg = PrepConfig(make_dataframe=False)
        paper_texts, _ = build_paper_texts(records, cfg)
        self.assertEqual(paper_texts[0], "123 ")

    def test_rejects_non_dict_records(self):
        records = [{"title": "OK", "abstract": "OK"}, "not_a_dict"]
        cfg = PrepConfig(make_dataframe=False)
        with self.assertRaises(TypeError):
            build_paper_texts(records, cfg)


class TestSection4BoW(unittest.TestCase):
    def test_build_bow_vectors_shape_and_sparse(self):
        texts = ["cat sat", "dog sat", "cat dog"]
        cfg = BoWConfig(name="t", lowercase=True, stop_words=None)
        vect, X = build_bow_vectors(texts, cfg)
        self.assertEqual(X.shape[0], 3)
        self.assertGreater(X.shape[1], 0)
        self.assertTrue(hasattr(X, "nnz"))
        self.assertTrue(hasattr(vect, "vocabulary_"))

    def test_lowercase_merges_tokens(self):
        texts = ["BERT model", "bert model"]

        vect_off, _ = build_bow_vectors(texts, BoWConfig(name="off", lowercase=False, stop_words=None))
        feats_off = set(vect_off.get_feature_names_out().tolist())
        self.assertIn("BERT", feats_off)
        self.assertIn("bert", feats_off)

        vect_on, _ = build_bow_vectors(texts, BoWConfig(name="on", lowercase=True, stop_words=None))
        feats_on = set(vect_on.get_feature_names_out().tolist())
        self.assertNotIn("BERT", feats_on)
        self.assertIn("bert", feats_on)

    def test_stopwords_removed_when_lowercase_true(self):
        texts = ["the cat sat", "the dog sat"]

        vect_no, _ = build_bow_vectors(texts, BoWConfig(name="no", lowercase=True, stop_words=None))
        feats_no = set(vect_no.get_feature_names_out().tolist())
        self.assertIn("the", feats_no)

        vect_yes, _ = build_bow_vectors(texts, BoWConfig(name="yes", lowercase=True, stop_words="english"))
        feats_yes = set(vect_yes.get_feature_names_out().tolist())
        self.assertNotIn("the", feats_yes)

    def test_case_sensitive_stopword_survivors_detected(self):
        texts = ["The cat and the dog"]
        vect, _ = build_bow_vectors(texts, BoWConfig(name="cs", lowercase=False, stop_words="english"))
        survivors = bow_case_sensitive_stopword_survivors(vect)
        self.assertGreaterEqual(survivors, 1)

    def test_sparse_stats_sane(self):
        texts = ["cat sat", "dog barked"]
        _, X = build_bow_vectors(texts, BoWConfig(name="s", lowercase=True, stop_words=None))
        s = bow_sparse_matrix_stats(X)
        self.assertEqual(s["n_docs"], 2)
        self.assertGreater(s["n_vocab"], 0)
        self.assertGreaterEqual(s["density"], 0.0)
        self.assertLessEqual(s["density"], 1.0)


class TestSection5Tfidf(unittest.TestCase):
    def test_build_tfidf_vectors_shape_sparse_and_float(self):
        texts = ["cat sat", "dog sat", "cat dog"]
        cfg = TfidfConfig(name="t", lowercase=True, stop_words=None)
        vect, X = build_tfidf_vectors(texts, cfg)
        self.assertEqual(X.shape[0], 3)
        self.assertGreater(X.shape[1], 0)
        self.assertTrue(hasattr(X, "nnz"))
        self.assertTrue("float" in str(X.dtype))

        stats = tfidf_sparse_matrix_stats(X)
        self.assertEqual(stats["n_docs"], 3)
        self.assertGreater(stats["n_vocab"], 0)

    def test_tfidf_lowercase_merges_tokens(self):
        texts = ["BERT model", "bert model"]

        vect_off, _ = build_tfidf_vectors(texts, TfidfConfig(name="off", lowercase=False, stop_words=None))
        feats_off = set(vect_off.get_feature_names_out().tolist())
        self.assertIn("BERT", feats_off)
        self.assertIn("bert", feats_off)

        vect_on, _ = build_tfidf_vectors(texts, TfidfConfig(name="on", lowercase=True, stop_words=None))
        feats_on = set(vect_on.get_feature_names_out().tolist())
        self.assertNotIn("BERT", feats_on)
        self.assertIn("bert", feats_on)

    def test_tfidf_stopwords_removed_when_lowercase_true(self):
        texts = ["the cat sat", "the dog sat"]

        vect_no, _ = build_tfidf_vectors(texts, TfidfConfig(name="no", lowercase=True, stop_words=None))
        feats_no = set(vect_no.get_feature_names_out().tolist())
        self.assertIn("the", feats_no)

        vect_yes, _ = build_tfidf_vectors(texts, TfidfConfig(name="yes", lowercase=True, stop_words="english"))
        feats_yes = set(vect_yes.get_feature_names_out().tolist())
        self.assertNotIn("the", feats_yes)

    def test_tfidf_case_sensitive_stopword_survivors_detected(self):
        texts = ["The cat and the dog"]
        vect, _ = build_tfidf_vectors(texts, TfidfConfig(name="cs", lowercase=False, stop_words="english"))
        survivors = tfidf_case_sensitive_stopword_survivors(vect)
        self.assertGreaterEqual(survivors, 1)

    def test_tfidf_l2_norms_are_about_one(self):
        texts = ["cat sat", "dog sat", "cat dog"]
        _, X = build_tfidf_vectors(texts, TfidfConfig(name="n", lowercase=True, stop_words=None))
        norms = l2_norm_sample_stats(X, sample_n=3, seed=0)
        self.assertAlmostEqual(norms["mean"], 1.0, places=6)

    def test_tfidf_idf_downweights_common_terms_when_tf_equal(self):
        texts = ["common rare1", "common rare2"]
        vect, X = build_tfidf_vectors(texts, TfidfConfig(name="idf", lowercase=True, stop_words=None))
        feats = vect.get_feature_names_out().tolist()

        i_common = feats.index("common")
        i_rare1 = feats.index("rare1")

        row0 = X.getrow(0).toarray().ravel()
        self.assertGreater(row0[i_rare1], row0[i_common])

    def test_top_terms_for_doc_returns_sorted(self):
        texts = ["cat sat sat", "dog sat"]
        vect, X = build_tfidf_vectors(texts, TfidfConfig(name="top", lowercase=True, stop_words=None))
        top = top_terms_for_doc(X, vect, doc_index=0, top_n=5)
        self.assertTrue(len(top) > 0)
        weights = [w for _, w in top]
        self.assertTrue(all(weights[i] >= weights[i + 1] for i in range(len(weights) - 1)))


class TestSection6Queries(unittest.TestCase):
    def test_default_queries_exist_and_valid(self):
        queries = get_default_queries(joiner=" ")
        validate_queries(queries)  # should not raise
        self.assertEqual(len(queries), 3)
        self.assertEqual([q.label for q in queries], ["Query 1", "Query 2", "Query 3"])
        self.assertTrue(all(isinstance(q, QueryItem) for q in queries))

    def test_query_texts_are_in_order(self):
        queries = get_default_queries(joiner=" ")
        texts = get_query_texts(queries)
        self.assertEqual(len(texts), 3)
        self.assertTrue(all(isinstance(t, str) and len(t) > 0 for t in texts))

    def test_build_query_item_strict_joiner(self):
        with self.assertRaises(ValueError):
            build_query_item("Query 1", "T", "A", joiner="  ")  # not exactly one space
        with self.assertRaises(ValueError):
            build_query_item("Query 1", "T", "A", joiner="")    # not exactly one space

    def test_validate_queries_rejects_wrong_order(self):
        q1 = build_query_item("Query 1", "T1", "A1", joiner=" ")
        q2 = build_query_item("Query 2", "T2", "A2", joiner=" ")
        q3 = build_query_item("Query 3", "T3", "A3", joiner=" ")
        with self.assertRaises(AssertionError):
            validate_queries([q2, q1, q3])  # wrong order/labels

    def test_validate_queries_rejects_bad_text_pipeline(self):
        bad = QueryItem(label="Query 1", title="T", abstract="A", text="T  A")
        q2 = build_query_item("Query 2", "T2", "A2", joiner=" ")
        q3 = build_query_item("Query 3", "T3", "A3", joiner=" ")
        with self.assertRaises(AssertionError):
            validate_queries([bad, q2, q3])


if __name__ == "__main__":
    unittest.main(verbosity=2)


Overwriting test_all.py


In [4]:
!python -m unittest -v test_all.py


test_basic_concatenation_rule (test_all.TestSection3Preprocessing.test_basic_concatenation_rule) ... ok
test_rejects_non_dict_records (test_all.TestSection3Preprocessing.test_rejects_non_dict_records) ... ok
test_stable_length_and_validation (test_all.TestSection3Preprocessing.test_stable_length_and_validation) ... ok
test_type_coercion_is_minimal_and_safe (test_all.TestSection3Preprocessing.test_type_coercion_is_minimal_and_safe) ... ok
test_build_bow_vectors_shape_and_sparse (test_all.TestSection4BoW.test_build_bow_vectors_shape_and_sparse) ... ok
test_case_sensitive_stopword_survivors_detected (test_all.TestSection4BoW.test_case_sensitive_stopword_survivors_detected) ... ok
test_lowercase_merges_tokens (test_all.TestSection4BoW.test_lowercase_merges_tokens) ... ok
test_sparse_stats_sane (test_all.TestSection4BoW.test_sparse_stats_sane) ... ok
test_stopwords_removed_when_lowercase_true (test_all.TestSection4BoW.test_stopwords_removed_when_lowercase_true) ... ok
test_build_tfidf_vecto

In [1]:
%%writefile section7_similarity_search.py
from __future__ import annotations

from dataclasses import dataclass
from typing import Any, Dict, List, Mapping, Optional, Sequence

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


@dataclass(frozen=True)
class RetrievedDoc:
    rank: int
    doc_index: int
    score: float
    text: str


def top_k_indices(scores: np.ndarray, k: int = 3) -> np.ndarray:
    """
    Return top-k indices sorted by:
      1) score descending
      2) index ascending (tie-break for determinism)
    """
    s = np.asarray(scores).ravel()
    if s.ndim != 1:
        raise ValueError("scores must be a 1D array")
    if k <= 0:
        raise ValueError("k must be >= 1")

    k_eff = int(min(k, s.size))
    # Fast candidate selection
    cand = np.argpartition(-s, k_eff - 1)[:k_eff]
    # Deterministic sorting: score desc, index asc
    order = np.lexsort((cand, -s[cand]))
    return cand[order]


def run_similarity_search(
    *,
    vectorizer: Any,
    paper_vectors: Any,
    paper_texts: Sequence[str],
    query_texts: Sequence[str],
    query_labels: Optional[Sequence[str]] = None,
    top_k: int = 3,
) -> Dict[str, Any]:
    """
    Core requirement for Section 7:
      - Use vectorizer.transform on queries (NOT fit_transform)
      - cosine similarity(query_vectors, paper_vectors)
      - retrieve top-k docs per query, returning their paper_texts[index] + score
    """
    if query_labels is None:
        query_labels = [f"Query {i+1}" for i in range(len(query_texts))]

    if len(query_texts) != len(query_labels):
        raise ValueError("query_texts and query_labels must have the same length")

    if not hasattr(paper_vectors, "shape"):
        raise TypeError("paper_vectors must be a matrix with .shape")

    n_docs = int(paper_vectors.shape[0])
    if len(paper_texts) != n_docs:
        raise ValueError(f"len(paper_texts)={len(paper_texts)} must match paper_vectors rows={n_docs}")

    # IMPORTANT: transform (re-use learned vocabulary)
    query_vectors = vectorizer.transform(list(query_texts))

    if int(query_vectors.shape[1]) != int(paper_vectors.shape[1]):
        raise RuntimeError(
            f"Vector dimension mismatch: query_vectors has {query_vectors.shape[1]} cols "
            f"but paper_vectors has {paper_vectors.shape[1]} cols"
        )

    sim = cosine_similarity(query_vectors, paper_vectors)  # (n_queries, n_docs) dense
    results: Dict[str, List[RetrievedDoc]] = {}

    for qi, qlabel in enumerate(query_labels):
        idxs = top_k_indices(sim[qi], k=top_k)
        matches: List[RetrievedDoc] = []
        for rank, di in enumerate(idxs, start=1):
            d = int(di)
            matches.append(
                RetrievedDoc(
                    rank=rank,
                    doc_index=d,
                    score=float(sim[qi, d]),
                    text=str(paper_texts[d]),
                )
            )
        results[str(qlabel)] = matches

    return {
        "query_vectors": query_vectors,
        "similarity_matrix": sim,
        "matches": results,
    }


def run_similarity_for_grid(
    *,
    grid_results: Mapping[str, Mapping[str, Any]],
    run_order: Sequence[str],
    representation_name: str,
    paper_texts: Sequence[str],
    query_texts: Sequence[str],
    query_labels: Sequence[str],
    top_k: int = 3,
) -> Dict[str, Any]:
    """
    Runs similarity search for each run inside bow_results / tfidf_results.
    Expects each grid_results[run_name] to contain keys: 'vectorizer', 'X', 'config' (optional).
    """
    out: Dict[str, Any] = {"representation": representation_name, "runs": {}}

    for run_name in run_order:
        if run_name not in grid_results:
            raise KeyError(f"run_name '{run_name}' not found in grid_results")

        run_obj = grid_results[run_name]
        vect = run_obj["vectorizer"]
        X = run_obj["X"]
        cfg = run_obj.get("config", None)

        res = run_similarity_search(
            vectorizer=vect,
            paper_vectors=X,
            paper_texts=paper_texts,
            query_texts=query_texts,
            query_labels=query_labels,
            top_k=top_k,
        )

        out["runs"][run_name] = {
            "config": cfg,
            "matches": res["matches"],
        }

    return out


def _truncate(text: str, max_chars: Optional[int]) -> str:
    if max_chars is None:
        return text
    t = str(text)
    return t if len(t) <= max_chars else (t[:max_chars] + "...")


def flatten_results(
    *,
    bow_block: Dict[str, Any],
    tfidf_block: Dict[str, Any],
    max_chars: Optional[int] = 350,
) -> List[Dict[str, Any]]:
    """
    Flatten nested results into rows (nice for pandas display).
    """
    rows: List[Dict[str, Any]] = []

    for block in (bow_block, tfidf_block):
        rep = block["representation"]
        for run_name, run_data in block["runs"].items():
            cfg = run_data.get("config", None)
            lowercase = getattr(cfg, "lowercase", None)
            stop_words = getattr(cfg, "stop_words", None)

            matches_by_query = run_data["matches"]
            for qlabel, matches in matches_by_query.items():
                for m in matches:
                    rows.append(
                        {
                            "representation": rep,
                            "run": run_name,
                            "lowercase": lowercase,
                            "stop_words": stop_words,
                            "query": qlabel,
                            "rank": m.rank,
                            "score": m.score,
                            "doc_index": m.doc_index,
                            "paper_text": _truncate(m.text, max_chars=max_chars),
                        }
                    )

    return rows


def results_to_dataframe(rows: List[Dict[str, Any]]):
    """
    Optional: convert rows to a pandas DataFrame for clean comparison in Colab.
    """
    try:
        import pandas as pd  # type: ignore
    except Exception:
        return None
    return pd.DataFrame(rows)


def print_results(
    *,
    bow_block: Dict[str, Any],
    tfidf_block: Dict[str, Any],
    max_chars: Optional[int] = 350,
) -> None:
    """
    Console-style report, organized to compare BoW vs TF-IDF and preprocessing toggles.
    """
    for block in (bow_block, tfidf_block):
        rep = block["representation"]
        print(f"\n==================== {rep} ====================")
        for run_name, run_data in block["runs"].items():
            cfg = run_data.get("config", None)
            lc = getattr(cfg, "lowercase", None)
            sw = getattr(cfg, "stop_words", None)

            print(f"\n--- {run_name} (lowercase={lc}, stop_words={sw}) ---")
            matches_by_query = run_data["matches"]
            for qlabel, matches in matches_by_query.items():
                print(f"\n{qlabel}:")
                for m in matches:
                    txt = _truncate(m.text, max_chars=max_chars)
                    print(f"{m.rank}. Text: '{txt}' (Score: {m.score:.4f})")


Writing section7_similarity_search.py


Use the following code snipet to compute the pairwise cosine similarity, and to sort the top 3 candidates for each query. In this example, `query_vectors` and `paper_vectors` correspond to the vectorized collections of queries and papers (as stored in the JSON file previously downloaded), respectively. The variable `paper_texts` contain the list of concatenated titles+abstracts of the papers loaded from the JSON file.

In [1]:
from sklearn.metrics.pairwise import cosine_similarity

#...

similarity_matrix = cosine_similarity(query_vectors, paper_vectors)

for nquery in [0, 1, 2]:
  similarity_scores = similarity_matrix[nquery]
  top_indices = np.argsort(similarity_scores)[::-1][:3]  # Indices of top 3 scores
  for i, index in enumerate(top_indices, 1):
    print(f"{i}. Text: '{paper_texts[index]}' (Score: {similarity_scores[index]:.4f})")
  print()

NameError: name 'query_vectors' is not defined

###Step 4: Analysis of the results obtained

## Part 2:

In this section students should use `setence-embeddings` to obtain sentence-embedding representations of the dataset and to peform searches for best matches regarding the examples proposed in the instructions of the assignment.

###Step 1: Trying a general purpose small monolingual model

###Step 2: Comparing other models

###Step 3: Moving to a multilingual environment

When trying multilingual models, you will have to build a multilingual collection of papers. To do so, extend the collection of papers from the EMNLP conference used in the first part of this notebook with an extra collection of papers from the SEPLN conference, which are both in English and Spanish. Create a new dataset that concatenates both collections to try a multilingual search. The collection of SEPLN papers can be downloaded from [https://www.dlsi.ua.es/~mespla/sepln.json](https://www.dlsi.ua.es/~mespla/sepln.json)

##Concluding remarks